# Viscolelasticity Example

In this notebook we provide an example of the DeepMoD algorithm by applying it to the stress response of a material as a result of a dynamic strain application. The data is synthetic and generated outside of this notebook.

We start by importing the required libraries and setting the plotting style:

In [2]:
import sys
import numpy as np
import torch

sys.path.append('../')
sys.path.append('../src')
from deepymod_torch.library_function import mech_library
from deepymod_torch.DeepMod import DeepMoD
from deepymod_torch.VE_params import coeffs_from_model_params, model_params_from_coeffs

Next we configure some known facts about the input manipulation. In this example we modelled a scenario when strain was the independant variable under our control, and the analytical expression defining this manipulation was a sinc function. The input function is written as a callable lambda function and uses PyTorch functions as the function will be called PyTorch tensors.

In [ ]:
Input_Type = 'Strain'
Input_Function = lambda time_data: torch.sin(time_data)/time_data

Next we load the data. The data file also has the strain data but for this method of implementation we can ignore it.

> Note: using this strain data directly will not allow us to calculated derivatives as PyTorch needs to track strain from teh orginal time data and know its history.

The data is placed into PyTorch tensors. As we will want to know derivatives with respect to time, we must flag that `time_Tensor` requires gradient tracking.

In [ ]:
data = np.loadtxt('../data/StressStrain/Strain Sinc E[1, 1, 1] V[1.25, 2.5].csv', delimiter=',')
time_Array, Stress_Array = data[:,0:1], data[:,2:]
time_Tensor = torch.tensor(time_Array, dtype=torch.float32, requires_grad=True)
Stress_Tensor = torch.tensor(Stress_Array, dtype=torch.float32)

Next we configure our parameters for the running of DeepMoD.

`optim_config` is rather standard.

In [ ]:
optim_config = {'lambda': 10**-5, 'max_iterations': 20000}

`network_config` indicates that we have one input data series (time) and one output data series (stress). The network is relatively large as we are trying to fit a semi-periodic function.

In [ ]:
network_config = {'input_dim': 1, 'hidden_dim': 50, 'layers': 4, 'output_dim': 1}

`lib_config` contains a large number of elements that require explanation.

- `mech_library` is the name given to the library function imported at the top of the notebook. This function will allow the calculation of the derivatives of strain and stress from the zeroth order through to the maximum order specified by `diff_order`. It also performs the organisation needed to place this data in $\Theta$.

- `coeff_sign` mandates that the initial guesses at the coefficients are all positive. The variability in the set of equations we are trying to fit allows for known signs on all the coefficients. Strain coefficients will be negative always, and stress positive, but as this is taken into account by the library function, DeepMoD is effectively only asked to find the magnitude of these values.

- The remaining two elements are simply the parameters we defined near the beginning.

In [ ]:
lib_config = {'type': mech_library, 'diff_order': 3, 'coeff_sign': 'positive', 'input_type': Input_Type, 'input_expr': Input_Function}

With this organisation completed, we can now use the DeepMoD neural network to solve the equation relating stress and strain.

Before doing so however, it useful to know the values of the coefficients that DeepMoD should find. We can do this using a function written for this purpose:

In [3]:
coeffs_from_model_params([1, 1, 1], [1.25, 2.5])

[0.266666666666667,
 0.833333333333333,
 0.800000000000000,
 2.00000000000000,
 0.833333333333333]

Now, let's run DeepMoD.

In [ ]:
sparse_coeff_vector, sparsity_mask, network = DeepMoD(time_Tensor, Stress_Tensor, network_config, lib_config, optim_config)

In [ ]:
sparse_coeff_vector

In [ ]:
sparsity_mask

To make sense of these coefficients and relate them to the best fit model paramters, we take only `sparse_coeff_vector` and submit this to a simultaneous equations solving algorithm written:

In [ ]:
Model_Params = model_params_from_coeffs(sparse_coeff_vector)
Model_Params